In [ ]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 80.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 105MB/s]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_df = pd.read_csv("train_data.csv")
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
test_df = pd.read_csv("test_data.csv")
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
print(train_df.shape, test_df.shape)

(1832, 111) (465, 111)


In [ ]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

# TF-IDF

In [ ]:
X_train_tfidf= X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_tfidf = X_test.drop(['clean_text'], axis=1).to_numpy()

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
tfidf.fit(pd.concat([X_train['clean_text'], X_test['clean_text']],ignore_index=True))

TfidfVectorizer()

In [ ]:
train_tfidf = tfidf.transform(X_train['clean_text']).toarray()

In [ ]:
X_train_tfidf = np.concatenate((X_train_tfidf, train_tfidf), axis=1)

In [ ]:
test_tfidf = tfidf.transform(X_test['clean_text']).toarray()

In [ ]:
X_test_tfidf = np.concatenate((X_test_tfidf, test_tfidf), axis=1)

In [ ]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(1832, 8262)
(465, 8262)


# Training-Testing Phase

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=0.4), scaler.fit_transform(X_train_tfidf), scaler.fit_transform(X_test_tfidf), y_train, y_test)

{'Accuracy': 0.789247311827957,
 'Precision': 0.7610294117647058,
 'Recall': 0.8625,
 'F1-Score': 0.80859375}

In [ ]:
training_and_prediction(GaussianNB(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.6580645161290323,
 'Precision': 0.6563706563706564,
 'Recall': 0.7083333333333334,
 'F1-Score': 0.6813627254509018}

In [ ]:
training_and_prediction(BernoulliNB(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7849462365591398,
 'Precision': 0.7573529411764706,
 'Recall': 0.8583333333333333,
 'F1-Score': 0.8046874999999999}

## SVM

In [ ]:
training_and_prediction(SVC(C=10), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.8021505376344086,
 'Precision': 0.7868217054263565,
 'Recall': 0.8458333333333333,
 'F1-Score': 0.8152610441767068}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.8021505376344086,
 'Precision': 0.7913385826771654,
 'Recall': 0.8375,
 'F1-Score': 0.8137651821862348}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(max_iter=5000, solver='saga'), X_train_tfidf, X_test_tfidf, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'Accuracy': 0.8064516129032258,
 'Precision': 0.8,
 'Recall': 0.8333333333333334,
 'F1-Score': 0.816326530612245}

## Gredient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.810752688172043,
 'Precision': 0.8166666666666667,
 'Recall': 0.8166666666666667,
 'F1-Score': 0.8166666666666667}

## Deep Learning Techniques

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

### ANN

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_tfidf, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_tfidf, y_test))
    y_pred_dense = model_dense.predict(X_test_tfidf)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [ ]:
for num_neurons_layer1 in [256, 128, 64]:
  for num_neurons_layer2 in [128, 64, 32]:
    for dropout_rate in [0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 8ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7956989247311828, 'Precision': 0.7911646586345381, 'Recall': 0.8208333333333333, 'F1-Score': 0.805725971370143}]
15/15 [==============================] - 0s 6ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7978494623655914, 'Precision': 0.7991803278688525, 'Recall': 0.8125, 'F1-Score': 0.8057851239669422}]
15/15 [==============================] - 0s 6ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.8064516129032258, 'Precision': 0.7737226277372263, 'Recall': 0.8833333333333333, 'F1-Score': 0.8249027237354085}]
15/15 [==============================] - 0s 6ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.4}, {'Accuracy': 0.7935483870967742, 'Precision': 0.7926829268292683, 'Recall': 0.8125, 'F1-Score': 0.8024